In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [176]:
import requests                #For downloading the HTML content using HTTP GET request
from bs4 import BeautifulSoup  #For parsing the HTML content and searching through the HTML
import os
import pandas as pd

In [3]:
#STAGE 1: extract all state URLs from the states page
url = "https://simple.wikipedia.org/wiki/List_of_U.S._states"
r = requests.get(url)
r.raise_for_status()

In [6]:
print(r.text)


<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of U.S. states - Simple English Wikipedia, the free encyclopedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XpJ7zApAIDAAAFNEJHMAAAAJ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_U.S._states","wgTitle":"List of U.S. states","wgCurRevisionId":6874248,"wgRevisionId":6874248,"wgArticleId":3023,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["States of the United States","Lists about U.S. states"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelevantPag

In [7]:
print(type(r.text))

<class 'str'>


In [9]:
doc = BeautifulSoup(r.text, "html.parser")
print(doc.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of U.S. states - Simple English Wikipedia, the free encyclopedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XpJ7zApAIDAAAFNEJHMAAAAJ","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_U.S._states","wgTitle":"List of U.S. states","wgCurRevisionId":6874248,"wgRevisionId":6874248,"wgArticleId":3023,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["States of the United States","Lists about U.S. states"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitex

In [13]:
tables = doc.find_all("table")
print(len(tables)) # only one table on the states page!
print(type(tables))
#Futuristic assert to make sure the html format hasn't changed on the website for the states page
assert len(tables) == 1 
tb1 = tables[0]

1
<class 'bs4.element.ResultSet'>


In [15]:
trs = tb1.find_all("tr")
print(len(trs))
assert len(trs) >= 50

52


In [154]:
prefix = "https://simple.wikipedia.org"
state_links = {} #KEY: state name; VALUE: link to state page

for tr in trs:
    th = tr.find("th")
    links = th.find_all("a")
    if len(links) == 0:
        continue
    link = links[0]
    state = link.get_text()
    #link.attrs is a dict
    state_url = prefix + link.attrs["href"]
    state_links[state] = state_url

In [159]:
#STAGE 2: download the html page for each state
html_dir = "html_files_for_states"
state_files = {} #KEY: state; VALUE: state file

for state in state_links.keys():
    if state == "postal abbreviation":
        continue
    state_url = state_links[state]

    #html file name
    state_file = os.path.join(html_dir, state + ".html")
    state_files[state] = state_file
    if os.path.exists(state_file):
        continue
    
    #Download
    r = requests.get(state_url)
    r.raise_for_status
    print(state_file)
    
    #Save to a file
    f = open(state_file, "w")
    f.write(r.text)
    f.close()

In [173]:
#STAGE 3: extract details from each state page

def state_stats(path):
    stats = {}
    f = open(path)
    html_string = f.read()
    f.close()
    
    doc = BeautifulSoup(html_string, "html.parser")
    trs = doc.find_all("tr")
    for tr in trs:
        cells = tr.find_all(["th", "td"])
        if len(cells) == 2:
            key = cells[0].get_text()
            value = cells[1].get_text()
            stats[key] = value
    return stats

wi_stats = state_stats(state_files["Wisconsin"])
print("WI state drink:", wi_stats["Beverage"])
print("WI state dance:", wi_stats["Dance"])

WI state drink: Milk
WI state dance: Polka


In [174]:
states_details = []

for state in state_files.keys():
    stats = state_stats(state_files[state])
    states_details.append(stats)

In [177]:
states_df = pd.DataFrame(states_details)
states_df

,,,Sly James(Kansas City) Francis G. Slay(St. Louis) Jim O'Neal(Springfield) Eileen Weir(Independence) Bob McDavid(Columbia),A,Admitted to the Union,Airport,Amphibian,Aqueducts andcanals,Area rank,Asheville Regional Airport,...,• Official language,• President of the Senate,• Rank,• Secretary of State,• Spoken language,• Summer (DST),• Total,• Upper house,• Water,• Width
0,Wikimedia Commons has media related to Alabama.,NaN,NaN,NaN,"December 14, 1819 (22nd)",NaN,Red Hills salamander,NaN,NaN,NaN,...,English,NaN,NaN,NaN,As of 2010[update][3]\nEnglish 95.1%\nSpanish ...,NaN,"4,863,300 (2,016 est.)[1]",Senate,NaN,NaN
1,Wikimedia Commons has media related to Alaska.,NaN,NaN,NaN,"January 3, 1959 (49th)",NaN,NaN,NaN,NaN,NaN,...,"English, Inupiat, Central Siberian Yupik, Cent...",NaN,NaN,NaN,English 86.3% Alaska Native languages 5.2%Taga...,NaN,"739,795 (2,017 est.)[1]",Senate,NaN,NaN
2,Wikimedia Commons has media related to Arizona.,NaN,NaN,NaN,"February 14, 1912 (48th)",NaN,Arizona tree frog,\nGrand Ditch\nColorado River Aqueduct\nSan Di...,NaN,NaN,...,English,NaN,NaN,Katie Hobbs (D),As of 2010\nEnglish 74.1%\nSpanish 19.5%\nNava...,NaN,"7,016,270 (2,017 est.)[1]",Senate,NaN,NaN
3,Wikimedia Commons has media related to Arkansas.,NaN,NaN,NaN,"June 15, 1836 (25th)",NaN,NaN,NaN,NaN,NaN,...,English,NaN,NaN,NaN,NaN,NaN,"3,004,279 (2,017 est.)[1]",Senate,NaN,NaN
4,Wikimedia Commons has media related to Califor...,NaN,NaN,NaN,"September 9, 1850 (31st)",NaN,California red-legged frog,\nGrand Ditch\nColorado River Aqueduct\nSan Di...,NaN,NaN,...,English,NaN,NaN,NaN,Language spoken at home[3]\nEnglish 58.1%\nSpa...,NaN,"39,557,045 (2,018 est.)[1]",Senate,NaN,NaN
5,Wikimedia Commons has media related to Colorado.,NaN,NaN,NaN,"August 1, 1876[1] (38th)",NaN,Western tiger salamanderAmbystoma mavortium,NaN,NaN,NaN,...,English,NaN,NaN,NaN,NaN,NaN,"5,695,564 (2,018 est.)[2]",Senate,NaN,NaN
6,Wikimedia Commons has media related to Connect...,NaN,NaN,NaN,"January 9, 1788 (5th)",NaN,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,NaN,NaN,"3,588,184 (2,017 est.)[3]",Connecticut Senate,NaN,NaN
7,NaN,Learning resources from Wikiversity\n,NaN,NaN,"December 7, 1787 (1st)",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"961,939 (2,017 est.)[1]",Senate,NaN,NaN
8,Wikimedia Commons has media related to Florida.,NaN,NaN,NaN,"March 3, 1845 (27th)",NaN,Barking tree frog,NaN,NaN,NaN,...,English[3],NaN,NaN,NaN,Predominantly English and Spanish[4],NaN,"21,299,325 (2,018 est.)[1]",Senate,NaN,NaN
9,Wikimedia Commons has media related to Georgia...,NaN,NaN,NaN,"January 2, 1788 (4th)",NaN,American green tree frog,NaN,NaN,NaN,...,English,NaN,NaN,NaN,"English, Spanish (7.42%)",NaN,"10,519,475 (est. 2,018)",State Senate,NaN,NaN
